# AEMET OpenData
Aemet pone a disposición de los interesados una interfaz amigable para hacer la descarga de todos los datos que oferta en OpenData (https://opendata.aemet.es/centrodedescargas/inicio).

En este notebook se muestra cómo se puede acceder a un subconjunto de los datos ofertados utilizando Python, bien de modo interactivo o utilizando el __módulo aemet_open_data__. Para trabajar en Python tienes que haber instalado la librería Pandas.

Los datos que puedes decargar utilizando este notebook son:
1. Estaciones climatológicas con datos en Aemet OpenData. Aemet tiene muchas más estaciones con datos, pero la mayor parte no están en OpenData.
1. Variables climatológicas diarias.
1. Variables climatológicas mensuales.

Desde Aemet puedes descargar las variables climatológicas, pero se ponen limitaciones en el número de datos a descargar por petición. Por ejemplo, para los datos diarios hay que hacer una petición por estación y un máximo de 5 años de serie; para los datos mensuales, una estación y un máximo de 36 meses. Si quieres obtener datos de más de una estación para una serie larga tienes que repetir el proceso. En estos casos es cuando utilizar el módulo aemet_open_data u otro similar te permite ahorrar tiempo.

Un cierto inconveniente es que entre los datos de las estaciones climatológicas OpenData no figura el primer dato de cada serie. Si quieres obtener las series completas tendrás que empezar haciendo peticiones desde una fecha inicial antigua e ir moviéndo progresivamente el rango hasta encontrar el inicio, lo que es tedioso si no utilizas un proceso automatizado, además de consumidor de recursos, pues en ocasiones tienes que hacer muchas peticiones que no van a devolver datos.  

Si no estás interesado en el modo interactivo y lo que quieres es disponer rápido de los datos, te recomiendo ir directamente al apartado de cómo se utiliza el módulo aemet_open_data.

En todos los casos necesitas una __clave de acceso__. Entra en la página de Aemet https://opendata.aemet.es/centrodedescargas/inicio y solicita una clave para operar en el servicio (APIKEY). La clave es una serie alfanumérica muy larga, sálvala en el fichero apikey.txt. Tu apikey no caduca.

## Python interactivo

Si tienes curiosidad de ver como se funciona de modo interactivo en Python.

Pasos:
1. Se hace una petición de datos a Aemet, para lo que se necesita formar una url específica de cada tipo de datos que queremos descargar.
1. Si la petición tiene éxito devuelve 2 url: una para descargar los datos y otra para descargar los metadatos. Estas 2 url tienen una tiempo de validez.


In [2]:
from datetime import date
import json
import pandas as pd
import requests

In [3]:
# Leo mi apikey
with open('apikey.txt') as f:
    myapikey = f.readline()

# Leo las urls que tengo preparadas (hay muchas más)
with open('urls.json', 'r') as f:
    urls = json.load(f)

querystring = {"api_key":myapikey}
headers = {'cache-control': "no-cache"}

# ver la url que teng preparadas y los parámetros (esta petición no tiene)aemet_open_data
urls.keys()

dict_keys(['estaciones', 'estacion_dia', 'estacion_mes'])

Es decir, en el fichero urls.json tengo guardadas 3 urls para descargar datos:
1. estaciones. Los datos de las estaciones
2. estacion_dia. Los datos de las variables climatológicas diarias por estación
3. estacion_mes. Los datos de las variables climatológicas mensuales por estación 

### Estaciones AEMET OpenData
Petición de todas las estaciones meteorológicas que sirven datos en el servicio OpenData 

In [10]:
# Para ver la información completa para descargar los datos de las estaciones
urls['estaciones']

['https://opendata.aemet.es/opendata/api/valores/climatologicos/inventarioestaciones/todasestaciones',
 []]

In [11]:
# veo que sólo necesito la url, es decir: 
urls['estaciones'][0]

'https://opendata.aemet.es/opendata/api/valores/climatologicos/inventarioestaciones/todasestaciones'

In [12]:
# aquí hago la petición (request) y veo la respuesta (response y content)
url = urls['estaciones'][0]
response = requests.request("GET", url, headers=headers, params=querystring)
content = json.loads(response.content)
content

{'descripcion': 'exito',
 'estado': 200,
 'datos': 'https://opendata.aemet.es/opendata/sh/5a60779b',
 'metadatos': 'https://opendata.aemet.es/opendata/sh/0556af7a'}

La petición me ha devuelto la respuesta del servidor de Aemet.

* Un código de la respuesta (estado) y una descripción del código
* Dos urls; una para los metadatos y otra para los datos. Estas urls están activas durante un tiempo limitado

In [13]:
# si la petición tiene éxito puedo descargar los datos; el formato original es json
# pero para majorar la visibilidad lo almaceno en una dataframe de pandas
df = pd.read_json(content["datos"], encoding='latin-1')
df.head()

,latitud,provincia,altitud,indicativo,nombre,indsinop,longitud
0,413515N,BARCELONA,74,0252D,ARENYS DE MAR,08186,023224E
1,411734N,BARCELONA,4,0076,BARCELONA AEROPUERTO,08181,020412E
2,412506N,BARCELONA,408,0200E,"BARCELONA, FABRA",,020727E
3,412326N,BARCELONA,6,0201D,BARCELONA,08180,021200E
4,414312N,BARCELONA,291,0149X,MANRESA,08174,015025E


In [14]:
# si la petición tuvo éxito también puedo descargar los metadatos
# en pantalla algunas columnas aparece truncadas, pero si las
# salvar a un fichero se graba el contenido completo
# observa que algunas columnas se visializan truncadas, pero si se graba
# en un fichero se puede ver el texto completo
df_meta = pd.read_json(content["metadatos"], encoding='latin-1')
df_meta.head()

,unidad_generadora,periodicidad,descripcion,formato,copyright,notaLegal,campos
0,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'latitud', 'descripcion': 'latitud de l..."
1,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'provincia', 'descripcion': 'provincia ..."
2,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'indicativo', 'descripcion': 'indicativ..."
3,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'altitud', 'descripcion': 'altitud de l..."
4,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'nombre', 'descripcion': 'ubicación de ..."


In [15]:
# Los datos en df y en df_meta los puedo salvar en un fichero con varios formatos
# aquí utilizo el formato csv, que puede ser leido con Excel o Calc, por ejemplo
# para guardar los datos (df) y sus metadatos (df_meta) cambia el nombre al fichero en dst 
# y en la instrucción to_csv pon df o df_meta según el caso
dst = '.\download\estaciones.csv'
df.to_csv(dst, index=False)

### Datos climatológicos diarios de una estación y un rango de fechas de hasta 5 años
Si quieres más de una estación o un periodo de más de 5 años hay que repetir el proceso para cada estación y periodo de hasta 5 años

In [16]:
# vemos la url a la que se va a hacer la petición y los parámetros:
# 2 fechas en un formato específico y un código de estación idema
urls['estacion_dia']

['https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/fechaIniStr/fechafin/fechaFinStr/estacion/idema',
 ['fechaIniStr format AAAA-MM-DDTHH:MM:SSUTC',
  'fechaFinStr format AAAA-MM-DDTHH:MM:SSUTC',
  'idema']]

La respuesta tiene la fefinición de la url y 3 parámetros que tengo que sustituir por un valor concreto en la petición:
* fechaIniStr y fechaFinStr, fechas iniciales de la petición de datos con formato AAAA-MM-DDTHH:MM:SSUTC
* idema, código de la estación

In [17]:
# Doy valor a una url que tengo preformada (url), defino los nombres de los parámetros (params)
# y asigno los valores de los parámetros
# Sólo puedo pedir los datos de una estación en un rango de fechas de menos de 5 años
# Si quiero más estaciones o un rango de fehas mayor hay que repetir el proceso cambiando
# el contenido de values
url = urls['estacion_dia'][0]
params = ('fechaIniStr', 'fechaFinStr', 'idema') 
values = ('2022-01-01T00:00:00UTC', '2023-01-01T23:59:59UTC', '7178I')

for par, value in zip(params, values):
    url = url.replace(par, value) 

print(url)

response = requests.request("GET", url, headers=headers, params=querystring)
response_json = response.json()
response_json

https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-01-01T00:00:00UTC/fechafin/2023-01-01T23:59:59UTC/estacion/7178I


{'descripcion': 'exito',
 'estado': 200,
 'datos': 'https://opendata.aemet.es/opendata/sh/d73b5665',
 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}

In [18]:
# Igual que antes: si la respuesta tiene éxito puedo descargar los datos
df = pd.read_json(response_json["datos"], encoding='latin-1')
df.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2022-01-01,7178I,MURCIA,MURCIA,62,"13,2","0,0","2,9",07:40,"23,5",15:10,30.0,"0,8","3,9",08:10,"8,6","1022,7",Varias,"1019,2",15
1,2022-01-02,7178I,MURCIA,MURCIA,62,"13,0","0,0","3,5",07:10,"22,6",15:20,30.0,"1,1","3,6",22:20,"8,0","1022,7",10,"1018,9",15
2,2022-01-03,7178I,MURCIA,MURCIA,62,"12,2","0,0","2,1",07:10,"22,4",15:40,23.0,"0,8","4,2",19:30,"8,7","1020,5",00,"1013,9",24
3,2022-01-04,7178I,MURCIA,MURCIA,62,"13,8","0,0","4,3",04:10,"23,3",14:30,29.0,"1,4","10,8",20:30,"8,5","1013,9",00,"1002,5",16
4,2022-01-05,7178I,MURCIA,MURCIA,62,"13,2","0,0","10,1",23:59,"16,2",14:30,28.0,"4,7","15,8",02:20,"5,5","1011,8",10,"1006,6",02


In [19]:
# y los metadatos
df_meta = pd.read_json(response_json["metadatos"], encoding='latin-1')
df_meta.head()

,unidad_generadora,periodicidad,descripcion,formato,copyright,notaLegal,campos
0,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'fecha', 'descripcion': 'fecha del dia ..."
1,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'indicativo', 'descripcion': 'indicativ..."
2,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'nombre', 'descripcion': 'nombre (ubica..."
3,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'provincia', 'descripcion': 'provincia ..."
4,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'altitud', 'descripcion': 'altitud de l..."


In [20]:
# y salvar los datos y/o los metadatos a fichero en formato csv
fo = '.\download\murcia_dia.csv'
df.to_csv(fo, index=False)

## Cómo se utiliza el módulo aemet_open_data
El módulo tiene utilidad sobre todo en los siguientes casos:
* Necesitamos pedir datos de varias estaciones
* El periodo de datos que necesitamos excede el establecido por Aemet para una petición

Es decir, cuando necesitamos datos de una o más estaciones y en periodos de más de 5 años para los datos diarios o 36 meses para datos mensuales.

En estos casos su utilización nos permite no tener que escribir varias veces la url con las peticiones y salvar cada una a un fichero; el módulo lo hace por nosotros.

Lo primero que hay que hacer son las importaciones:

In [10]:
# Lo más simple es tener el notebook en el directorio donde has guardado el fichero aemet_open_data.py
# hay otras técnnicas que no se consideran aquí
from datetime import date
import inspect
import pandas as pd
from pathlib import Path

from aemet_open_data import AemetOpenData

# instanciación de un objeto AemetOpenData
# el objeto lee la clave de acceso del fichero apikey.txt
# que está situado en el mismo directorio que el notebook y aemet_open_data.py
aod = AemetOpenData()

### Estaciones con datos de variables cimatológicas en OpenData
Vamos a ver la documentación de la función estaciones_climatologicas:

In [9]:
# Documentación de la función que vamos a utilizar estaciones_climatologicas
print(aod.estaciones_climatologicas.__doc__)


        Retrieves data from meteorological stations using Aemet OpenData and
            saves them to a file

        Parameters
        ----------
        dir_out: The name of the directory where the data file will be saved
        metadata: If True only metadata will be saved, otherwise only data
        will be saved

        Returns
        -------
        file name with data or metadata
        


In [4]:
# Parámetros
parameters = inspect.signature(aod.estaciones_climatologicas).parameters
print("Parámetros de la función:")
for name, param in parameters.items():
    print(f"{name}: {param.annotation}")

Parámetros de la función:
dir_out: <class 'str'>
metadata: <class 'bool'>


In [7]:
# damos valor a los parámetros
# en dir_out puedes poner cualquier nombre de directorio con permisos de escritura
# si trabajas en Windows pon una r delante, por ej. r'C:\Users\user1\Documents\tmp'
# si no pones la r delante tienes que escribirlo así: 'C:\\Users\\user1\\Documents\\tmp'  
dir_output = './download'

# si metadata es True descarga los metadatos; si es False descarga los datos 
# prueba a asignar a la variable metadata los valores True o False
metadata = False

file_name = aod.estaciones_climatologicas(dir_output, metadata = metadata)
print(f'Metadatos: {metadata}, nombre de fichero: {file_name}')

Downloaded data in estaciones_open_data.csv
Metadatos: False, nombre de fichero: estaciones_open_data.csv


In [12]:
# Para ver los datos descargados puedes utilizar un programa que abra ficheros csv,
# por ejemplo un editor de texto plano, Excel, Calc u otros.
# Desde este notebook podemos utilizar la siguiente instrucción
f = Path(dir_output).joinpath(file_name)
df = pd.read_csv(f)
# vemos las 5 primeras líneas
df.head()
# para ver las 5 últimas escribimos df.tail()
# se puede ver el fichero entero, pero no nos distraemos con esto

,latitud,provincia,altitud,indicativo,nombre,indsinop,longitud
0,413515N,BARCELONA,74,0252D,ARENYS DE MAR,8186.0,023224E
1,411734N,BARCELONA,4,0076,BARCELONA AEROPUERTO,8181.0,020412E
2,412506N,BARCELONA,408,0200E,"BARCELONA, FABRA",NaN,020727E
3,412326N,BARCELONA,6,0201D,BARCELONA,8180.0,021200E
4,414312N,BARCELONA,291,0149X,MANRESA,8174.0,015025E


### Datos climatológicos diarios o mensuales
La descarga de los datos climatológicos diarios o mensuales es lo que principalmente justifica la utilitilización del módulo.

Al igual que la función anterior, vamos a ver la documentación de la función y los parámetros que utiliza

In [13]:
# Documentación de la función que vamos a utilizar variables_clima_estacion
print(aod.variables_clima_estacion.__doc__)


        Retrieves daily or monthly climatological data from Aemet OpenData
            server.

        Parameters
        ----------
        var_type : day for daily variables, month for monthly ones
        d1 : Initial date (daily data) or year (monthly data)
        d2 : Final date or year. Equal d1
        estaciones : List of stations or only one station as str
        dir_out : Each request to Aemet server is saved as a csv file in the
            directory dir_out        
        metadata: If True only metadata are downloaded, otherwise only data
            are downloaded
        verbose: If True shows detailed information of results request in the
            screen
        use_files : If True checks that the file name already exists in dir_out
            and does not make the request to the server; otherwise the request
            is made and the pre-existing file is overwritten.
        Returns
        -------
        [] with the the names of saved csv files. Each file h

In [14]:
# Parámetros de variables_clima_estacion
parameters = inspect.signature(aod.variables_clima_estacion).parameters
print("Parámetros de la función:")
for name, param in parameters.items():
    print(f"{name}: {param.annotation}")

Parámetros de la función:
var_type: <class 'str'>
d1: <class 'datetime.date'>
d2: <class 'datetime.date'>
estaciones: typing.Union[tuple[str], list[str], str]
dir_out: <class 'str'>
metadata: <class 'bool'>
verbose: <class 'bool'>
use_files: <class 'bool'>


Necesito definir la fecha inicial, la fecha final y la estación o estaciones de las que quiero los datos

El código de la estación lo puedes consultar de la consulta previa de estaciones. 

Aemet tiene muchas más estaciones, pero sólo algunas están activas en OpenData.

En la siguiente celda se definen los parámetros de la petición de datos. Internamente, una petición de datos puede ser resuelta con varias peticiones a Aemet; los resultados de cada una de estas peticiones internas a Aemet se graban en un fichero por si se produce un error inesperado no tener que empezar otra vez desde cero. 